# Information Theory Measures w/ RBIG

In [ ]:
!pip install "git+https://github.com/IPL-UV/rbig.git"

  Cloning https://github.com/IPL-UV/rbig.git to /tmp/pip-req-build-2khojwyv
  Running command git clone --filter=blob:none --quiet https://github.com/IPL-UV/rbig.git /tmp/pip-req-build-2khojwyv
  Resolved https://github.com/IPL-UV/rbig.git to commit a9124fc07d8e851833802d604fe552b6327ab980
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.5 MB/s eta 0:00:00
  Created wheel for py-rbig: filename=py_rbig-0.0.1-py3-none-any.whl size=17469 sha256=d9642c7de4392d0d450af3daf5e69d7c57a4fa5359ec167167415fe2917f3029
  Stored in directory: /tmp/pip-ephem-wheel-cache-f92lulid/wheels/49/be/80/0285fa4ec7c96e2c9a8c8bcb0cd0330a26e0ce7fbb4b5afc19
Successfully built py-rbig


In [ ]:
import sys

import numpy as np
import warnings
from time import time
#from rbig.rbig import RBIGKLD, RBIG, RBIGMI, entropy_marginal
from rbig import RBIG, entropy_marginal

from rbig._src.mutual_info import MutualInfoRBIG
#from rbig._src.mutual_info import MutualInfoRBIG

from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

warnings.filterwarnings('ignore') # get rid of annoying warnings

%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.10/dist-packages/rbig/_src/training.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


---
## Total Correlation

In [ ]:
#Parameters
n_samples = 10000
d_dimensions = 10

seed = 123


rng = check_random_state(seed)

#### Sample Data

In [ ]:
# Generate random normal data
data_original = rng.randn(n_samples, d_dimensions)

# Generate random Data
A = rng.rand(d_dimensions, d_dimensions)

data = data_original @ A

# covariance matrix
C = A.T @ A
vv = np.diag(C)

#### Calculate Total Correlation

In [ ]:
tc_original = np.log(np.sqrt(vv)).sum() - 0.5 * np.log(np.linalg.det(C))

print(f"TC: {tc_original:.4f}")

TC: 9.9326


### RBIG - TC

In [ ]:
%%time
n_layers = 10000
rotation_type = 'PCA'

#RBIG = RBIG()

# Initialize RBIG class
tc_rbig_model = RBIG(max_layers=n_layers,
                  rotation=rotation_type,
                  )

# fit model to the data
tc_rbig_model.fit(data);

  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 8.77 s, sys: 5.7 s, total: 14.5 s
Wall time: 13.7 s


RBIG(max_layers=10000)

In [ ]:
tc_rbig = tc_rbig_model.total_correlation() * np.log(2)
print(f"TC (RBIG): {tc_rbig:.4f}")
print(f"TC: {tc_original:.4f}")

TC (RBIG): 6.8649
TC: 9.9326


---
## Entropy

#### Sample Data

In [ ]:
#Parameters
n_samples = 5000
d_dimensions = 10

seed = 123

rng = check_random_state(seed)

# Generate random normal data
data_original = rng.randn(n_samples, d_dimensions)

# Generate random Data
A = rng.rand(d_dimensions, d_dimensions)

data = data_original @ A


#### Calculate Entropy

In [ ]:
Hx = entropy_marginal(data)

H_original = Hx.sum() + np.log2(np.abs(np.linalg.det(A)))

H_original *= np.log(2)

print(f"H: {H_original:.4f}")

H: 10.3427


### Entropy RBIG

In [ ]:
%%time
n_layers = 10000
rotation_type = 'PCA'

# Initialize RBIG class
ent_rbig_model = RBIG(max_layers=n_layers,
                  rotation=rotation_type,
                  )

# fit model to the data
ent_rbig_model.fit(data);

  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 7.44 s, sys: 4.19 s, total: 11.6 s
Wall time: 12.9 s


RBIG(max_layers=10000)

In [ ]:
Hx = entropy_marginal(data)
H_rbig = Hx.sum() - ent_rbig_model.total_correlation() * np.log(2)
print(f"Entropy (RBIG): {H_rbig:.4f}")
print(f"Entropy: {H_original:.4f}")

Entropy (RBIG): 13.5708
Entropy: 10.3427


---
## Mutual Information

#### Sample Data

In [ ]:
#Parameters
n_samples = 10000
d_dimensions = 10

seed = 123

rng = check_random_state(seed)

# Generate random Data
A = rng.rand(2 * d_dimensions, 2 * d_dimensions)

# Covariance Matrix
C = A @ A.T
mu = np.zeros((2 * d_dimensions))

dat_all = rng.multivariate_normal(mu, C, n_samples)

CX = C[:d_dimensions, :d_dimensions]
CY = C[d_dimensions:, d_dimensions:]

X = dat_all[:, :d_dimensions]
Y = dat_all[:, d_dimensions:]

#### Calculate Mutual Information

In [ ]:
H_X = 0.5 * np.log(2 * np.pi * np.exp(1) * np.abs(np.linalg.det(CX)))
H_Y = 0.5 * np.log(2 * np.pi * np.exp(1) * np.abs(np.linalg.det(CY)))
H = 0.5 * np.log(2 * np.pi * np.exp(1) * np.abs(np.linalg.det(C)))

mi_original = H_X + H_Y - H
mi_original *= np.log(2)

print(f"MI: {mi_original:.4f}")

MI: 8.0713


### RBIG - Mutual Information

In [ ]:
%%time
n_layers = 10000
rotation_type = 'PCA'


# Initialize RBIG class
rbig_model = MutualInfoRBIG(max_layers=n_layers,
                  rotation=rotation_type,
                  )

# fit model to the data
rbig_model.fit(X, Y);

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 31.1 s, sys: 23.2 s, total: 54.3 s
Wall time: 35.7 s


In [ ]:
mi_rbig = rbig_model.mutual_info() * np.log(2)

print(f"MI (RBIG): {mi_rbig:.4f}")
print(f"MI: {mi_original:.4f}")

MI (RBIG): 6.5699
MI: 8.0713
